In [22]:
import pandas as pd
import censusdata

In [23]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

In [24]:
censusdata.search('acs1', 2012, 'concept', 'race')

[('B01001F_001E', 'SEX BY AGE (SOME OTHER RACE ALONE)', 'Estimate!!Total'),
 ('B01001F_002E',
  'SEX BY AGE (SOME OTHER RACE ALONE)',
  'Estimate!!Total!!Male'),
 ('B01001F_003E',
  'SEX BY AGE (SOME OTHER RACE ALONE)',
  'Estimate!!Total!!Male!!Under 5 years'),
 ('B01001F_004E',
  'SEX BY AGE (SOME OTHER RACE ALONE)',
  'Estimate!!Total!!Male!!5 to 9 years'),
 ('B01001F_005E',
  'SEX BY AGE (SOME OTHER RACE ALONE)',
  'Estimate!!Total!!Male!!10 to 14 years'),
 ('B01001F_006E',
  'SEX BY AGE (SOME OTHER RACE ALONE)',
  'Estimate!!Total!!Male!!15 to 17 years'),
 ('B01001F_007E',
  'SEX BY AGE (SOME OTHER RACE ALONE)',
  'Estimate!!Total!!Male!!18 and 19 years'),
 ('B01001F_008E',
  'SEX BY AGE (SOME OTHER RACE ALONE)',
  'Estimate!!Total!!Male!!20 to 24 years'),
 ('B01001F_009E',
  'SEX BY AGE (SOME OTHER RACE ALONE)',
  'Estimate!!Total!!Male!!25 to 29 years'),
 ('B01001F_010E',
  'SEX BY AGE (SOME OTHER RACE ALONE)',
  'Estimate!!Total!!Male!!30 to 34 years'),
 ('B01001F_011E',
  'SEX

In [25]:
censusdata.search('acs1', 2012, 'label', 'age')[0]

('B01002A_001E',
 'MEDIAN AGE BY SEX (WHITE ALONE)',
 'Estimate!!Median age!!Total')

In [26]:
censusdata.printtable(censusdata.censustable('acs1', 2012, 'B01001'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B01001_001E  | SEX BY AGE                     | !! Estimate Total                                        | int  
B01001_002E  | SEX BY AGE                     | !! !! Estimate Total Male                                | int  
B01001_003E  | SEX BY AGE                     | !! !! !! Estimate Total Male Under 5 years               | int  
B01001_004E  | SEX BY AGE                     | !! !! !! Estimate Total Male 5 to 9 years                | int  
B01001_005E  | SEX BY AGE                     | !! !! !! Estimate Total Male 10 to 14 years              | int  
B01001_006E  | SEX BY AGE                     | !! !! !! Estimate Total Male 15 to 17 years              | int  
B01001_007E  | SEX BY AGE                     | !! !! !! Estimate Total Male 18 and 19 years 

In [27]:
censusdata.printtable(censusdata.censustable('acs1', 2012, 'B01002A'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B01002A_001E | MEDIAN AGE BY SEX (WHITE ALONE | !! !! Estimate Median age Total                          | float
B01002A_002E | MEDIAN AGE BY SEX (WHITE ALONE | !! !! Estimate Median age Male                           | float
B01002A_003E | MEDIAN AGE BY SEX (WHITE ALONE | !! !! Estimate Median age Female                         | float
-------------------------------------------------------------------------------------------------------------------


In [28]:
censusdata.geographies(censusdata.censusgeo([('state', '*')]), 'acs1', 2012)

{'Alabama': censusgeo((('state', '01'),)),
 'Alaska': censusgeo((('state', '02'),)),
 'Arizona': censusgeo((('state', '04'),)),
 'Arkansas': censusgeo((('state', '05'),)),
 'California': censusgeo((('state', '06'),)),
 'Colorado': censusgeo((('state', '08'),)),
 'Delaware': censusgeo((('state', '10'),)),
 'District of Columbia': censusgeo((('state', '11'),)),
 'Connecticut': censusgeo((('state', '09'),)),
 'Florida': censusgeo((('state', '12'),)),
 'Georgia': censusgeo((('state', '13'),)),
 'Idaho': censusgeo((('state', '16'),)),
 'Hawaii': censusgeo((('state', '15'),)),
 'Illinois': censusgeo((('state', '17'),)),
 'Indiana': censusgeo((('state', '18'),)),
 'Iowa': censusgeo((('state', '19'),)),
 'Kansas': censusgeo((('state', '20'),)),
 'Kentucky': censusgeo((('state', '21'),)),
 'Louisiana': censusgeo((('state', '22'),)),
 'Maine': censusgeo((('state', '23'),)),
 'Maryland': censusgeo((('state', '24'),)),
 'Massachusetts': censusgeo((('state', '25'),)),
 'Michigan': censusgeo((('stat

In [30]:
censusdata.geographies(censusdata.censusgeo([('state', '36'), ('state%20legislative%20district%20(upper%20chamber)', '*')]), 'acs5', 2012)


{'State Senate District 40 (2012), New York': censusgeo((('state', '36'), ('state legislative district (upper chamber)', '040'))),
 'State Senate District 8 (2012), New York': censusgeo((('state', '36'), ('state legislative district (upper chamber)', '008'))),
 'State Senate District 39 (2012), New York': censusgeo((('state', '36'), ('state legislative district (upper chamber)', '039'))),
 'State Senate District 9 (2012), New York': censusgeo((('state', '36'), ('state legislative district (upper chamber)', '009'))),
 'State Senate District 10 (2012), New York': censusgeo((('state', '36'), ('state legislative district (upper chamber)', '010'))),
 'State Senate District 57 (2012), New York': censusgeo((('state', '36'), ('state legislative district (upper chamber)', '057'))),
 'State Senate District 58 (2012), New York': censusgeo((('state', '36'), ('state legislative district (upper chamber)', '058'))),
 'State Senate District 59 (2012), New York': censusgeo((('state', '36'), ('state leg

In [31]:
data = censusdata.download('acs5', 2012, censusdata.censusgeo([('state', '36'), ('state%20legislative%20district%20(upper%20chamber)', '*')]), ['B01001_001E', 'B01001_002E', 'B01001_026E'])

In [32]:
data

,B01001_001E,B01001_002E,B01001_026E
"State Senate District 40 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):040",304418,150390,154028
"State Senate District 8 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):008",312213,150938,161275
"State Senate District 39 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):039",293597,145730,147867
"State Senate District 9 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):009",312806,151273,161533
"State Senate District 10 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):010",323789,151850,171939
...,...,...,...
"State Senate District 34 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):034",321252,157436,163816
"State Senate District 35 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):035",301542,145250,156292
"State Senate District 36 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):036",325191,146459,178732
"State Senate District 37 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):037",314509,151213,163296


In [33]:
values_to_grab = {'B01001001': "Total Pop",
                  'B01001002': "Male",
                  'B01001026': "Female",
                  'B02001002':"White alone",
                  'B02001003':"Black or African American alone",
                  'B02001004':"American Indian and Alaska Native alone",
                  'B02001005':"Asian alone",
                  'B02001006':"Native Hawaiian and Other Pacific Islander alone",
                  'B02001007':"Some other race alone",
                  'B02001008':"Two or more races:",
                  'B02001009':"Two races including Some other race",
                  'B02001010':"Two races excluding Some other race, and three or more races",
                  'B05001002':"U.S. citizen, born in the United States",
                  'B05001003':"U.S. citizen, born in Puerto Rico or U.S. Island Areas",
                  # Universe is limited to families with children
                  # 'B05010002':"Under poverty level",
                  # 'B05010010':"Between 1x and 1.99x poverty level",
                  # 'B05010018':"At least 2x poverty level",
                  "B06007002"	:"Speak only English",
                  "B06007003"	:"Speak Spanish",
                  "B06007006"	:"Speak other languages",
                  "B06008002"	:"Never married",
                  "B06008003"	:"Now married, except separated",
                  "B06008004"	:"Divorced",
                  "B06008005"	:"Separated",
                  "B06008006"	:"Widowed",
                  "B06009002"	:"Less than high school graduate",
                  "B06009003"	:"High school graduate (includes equivalency)",
                  "B06009004"	:"Some college or associate's degree",
                  "B06009005"	:"Bachelor's degree",
                  "B06009006"	:"Graduate or professional degree",
                  "B06010002"	:"No income",
                  "B06010004"	:"$1 to $9,999 or loss",
                  "B06010005"	:"$10,000 to $14,999",
                  "B06010006"	:"$15,000 to $24,999",
                  "B06010007"	:"$25,000 to $34,999",
                  "B06010008"	:"$35,000 to $49,999",
                  "B06010009"	:"$50,000 to $64,999",
                  "B06010010"	:"$65,000 to $74,999",
                  "B06010011"	:"$75,000 or more",
                  "B16004002"	:"5 to 17 years",
                  "B16004024"	:"18 to 64 years",
                  "B16004046"	:"65 years and over"








}

In [34]:
ny_senate_demo_data_2012 = censusdata.download('acs5', 2012, censusdata.censusgeo([('state', '36'), ('state%20legislative%20district%20(upper%20chamber)', '*')]), [each[:6] + '_' + each[6:] + 'E' for each in values_to_grab.keys()])

In [35]:
ny_senate_demo_data_2012 = ny_senate_demo_data_2012.rename(columns=dict(zip([each[:6] + '_' + each[6:] + 'E' for each in values_to_grab.keys()], values_to_grab.values())))

ny_senate_demo_data_2012

,Total Pop,Male,Female,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,Two or more races:,...,"$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $64,999","$65,000 to $74,999","$75,000 or more",5 to 17 years,18 to 64 years,65 years and over
"State Senate District 40 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):040",304418,150390,154028,253024,15252,622,10588,27,17849,7056,...,14996,25047,21186,23325,20229,10449,62704,57746,188432,42036
"State Senate District 8 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):008",312213,150938,161275,227510,50463,934,9875,9,16133,7289,...,17663,27642,24005,29723,23567,11550,48856,55467,195262,43341
"State Senate District 39 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):039",293597,145730,147867,229963,29964,689,7720,20,18671,6570,...,18094,27908,22690,24429,21008,10466,35704,59077,180984,31151
"State Senate District 9 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):009",312806,151273,161533,232591,37889,644,18038,9,17437,6198,...,16376,26816,21828,27145,25529,11300,57567,55381,194032,47158
"State Senate District 10 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):010",323789,151850,171939,48425,169970,1496,40305,141,50220,13232,...,20006,33351,28865,31572,20248,7768,18488,61054,209064,32153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"State Senate District 34 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):034",321252,157436,163816,161872,48047,1058,22331,0,77737,10207,...,19058,29218,24044,31574,23043,9317,33237,48533,204034,49978
"State Senate District 35 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):035",301542,145250,156292,175189,57545,918,21284,66,36125,10415,...,17887,27945,22304,24275,18990,8756,48501,53054,186220,42745
"State Senate District 36 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):036",325191,146459,178732,47342,198884,1297,10095,48,57713,9812,...,20781,34816,29242,34730,21409,8217,16314,58280,204993,39735
"State Senate District 37 (2012), New York: Summary level: 610, state:36> state legislative district (upper chamber):037",314509,151213,163296,239728,20967,402,17666,35,28430,7281,...,15862,25487,21038,24365,22245,10722,61741,55424,190926,49611


In [36]:
ny_senate_demo_data_2012 = ny_senate_demo_data_2012.reset_index(drop=False)

ny_senate_demo_data_2012

,index,Total Pop,Male,Female,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,...,"$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $64,999","$65,000 to $74,999","$75,000 or more",5 to 17 years,18 to 64 years,65 years and over
0,"State Senate District 40 (2012), New York: Sum...",304418,150390,154028,253024,15252,622,10588,27,17849,...,14996,25047,21186,23325,20229,10449,62704,57746,188432,42036
1,"State Senate District 8 (2012), New York: Summ...",312213,150938,161275,227510,50463,934,9875,9,16133,...,17663,27642,24005,29723,23567,11550,48856,55467,195262,43341
2,"State Senate District 39 (2012), New York: Sum...",293597,145730,147867,229963,29964,689,7720,20,18671,...,18094,27908,22690,24429,21008,10466,35704,59077,180984,31151
3,"State Senate District 9 (2012), New York: Summ...",312806,151273,161533,232591,37889,644,18038,9,17437,...,16376,26816,21828,27145,25529,11300,57567,55381,194032,47158
4,"State Senate District 10 (2012), New York: Sum...",323789,151850,171939,48425,169970,1496,40305,141,50220,...,20006,33351,28865,31572,20248,7768,18488,61054,209064,32153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,"State Senate District 34 (2012), New York: Sum...",321252,157436,163816,161872,48047,1058,22331,0,77737,...,19058,29218,24044,31574,23043,9317,33237,48533,204034,49978
59,"State Senate District 35 (2012), New York: Sum...",301542,145250,156292,175189,57545,918,21284,66,36125,...,17887,27945,22304,24275,18990,8756,48501,53054,186220,42745
60,"State Senate District 36 (2012), New York: Sum...",325191,146459,178732,47342,198884,1297,10095,48,57713,...,20781,34816,29242,34730,21409,8217,16314,58280,204993,39735
61,"State Senate District 37 (2012), New York: Sum...",314509,151213,163296,239728,20967,402,17666,35,28430,...,15862,25487,21038,24365,22245,10722,61741,55424,190926,49611


In [37]:
# u for upper, l for lower
ny_senate_demo_data_2012['index'] = ny_senate_demo_data_2012['index'].apply(lambda x: 'u'+x.geo[1][1])
ny_senate_demo_data_2012 = ny_senate_demo_data_2012.rename(columns={'index': 'district'})
ny_senate_demo_data_2012

,district,Total Pop,Male,Female,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some other race alone,...,"$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $64,999","$65,000 to $74,999","$75,000 or more",5 to 17 years,18 to 64 years,65 years and over
0,u040,304418,150390,154028,253024,15252,622,10588,27,17849,...,14996,25047,21186,23325,20229,10449,62704,57746,188432,42036
1,u008,312213,150938,161275,227510,50463,934,9875,9,16133,...,17663,27642,24005,29723,23567,11550,48856,55467,195262,43341
2,u039,293597,145730,147867,229963,29964,689,7720,20,18671,...,18094,27908,22690,24429,21008,10466,35704,59077,180984,31151
3,u009,312806,151273,161533,232591,37889,644,18038,9,17437,...,16376,26816,21828,27145,25529,11300,57567,55381,194032,47158
4,u010,323789,151850,171939,48425,169970,1496,40305,141,50220,...,20006,33351,28865,31572,20248,7768,18488,61054,209064,32153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,u034,321252,157436,163816,161872,48047,1058,22331,0,77737,...,19058,29218,24044,31574,23043,9317,33237,48533,204034,49978
59,u035,301542,145250,156292,175189,57545,918,21284,66,36125,...,17887,27945,22304,24275,18990,8756,48501,53054,186220,42745
60,u036,325191,146459,178732,47342,198884,1297,10095,48,57713,...,20781,34816,29242,34730,21409,8217,16314,58280,204993,39735
61,u037,314509,151213,163296,239728,20967,402,17666,35,28430,...,15862,25487,21038,24365,22245,10722,61741,55424,190926,49611


In [21]:
ny_senate_demo_data_2012['index'][0]

'u040'

In [ ]:
dir(ny_senate_demo_data_2012['index'][0])

In [38]:
harvard_data = pd.read_csv('196slers1967to2016_20180908.tab', delimiter='\t')

c:\users\stone\pycharmprojects\pythonproject1\venv\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,9,12,13,17,22,27,31,39,40,43,47,55,66,70,71,74,75,76,77,78,79,80,81,82,83,84,85,86,87,91) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [39]:
harvard_data[(harvard_data['sab'] == 'ny') & (harvard_data['year'] >= 2011)]['ddez']

238120     1
238121     1
238122     1
238123     2
238124     2
          ..
248553    63
248554    63
248555    63
248556    63
248557    63
Name: ddez, Length: 5110, dtype: object

In [56]:
ny_election_data = harvard_data[(harvard_data['sab'] == 'ny') & (harvard_data['year'] >= 2011)][['year', 'sab', 'ddez', 'sen', 'dname', 'dno', 'dtype', 'etype', 'cando', 'cand', 'party', 'prior1', 'prior2', 'exper', 'vote', 'outcome']]

ny_election_data

,year,sab,ddez,sen,dname,dno,dtype,etype,cando,cand,party,prior1,prior2,exper,vote,outcome
238120,2012,ny,1,0,NaN,1.0,1.0,g,NaN,"thiele, fred w. jr.",democrat,NaN,NaN,inc,29644.0,w
238121,2012,ny,1,0,NaN,1.0,1.0,g,NaN,"thiele, fred w. jr.",workingfamilies,NaN,NaN,inc,4142.0,w
238122,2012,ny,1,0,NaN,1.0,1.0,g,NaN,"thiele, fred w. jr.",independent,NaN,NaN,inc,4219.0,w
238123,2012,ny,2,0,NaN,2.0,1.0,g,NaN,"deegan, nicholas f.",democrat,NaN,NaN,none,18159.0,l
238124,2012,ny,2,0,NaN,2.0,1.0,g,NaN,"losquadro, daniel p.",independent,NaN,NaN,inc,1941.0,w
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248553,2016,ny,63,1,NaN,63.0,1.0,g,Timothy M. Kennedy,"kennedy, timothy m.",democrat,NaN,NaN,inc,78674.0,w
248554,2016,ny,63,1,NaN,63.0,1.0,g,Timothy M. Kennedy,"kennedy, timothy m.",wep,NaN,NaN,inc,1191.0,w
248555,2016,ny,63,1,NaN,63.0,1.0,g,Timothy M. Kennedy,"kennedy, timothy m.",indep,NaN,NaN,inc,3751.0,w
248556,2016,ny,63,1,NaN,63.0,1.0,g,Timothy M. Kennedy,"kennedy, timothy m.",working,NaN,NaN,inc,6034.0,w


In [58]:
ny_election_data['prior1'].value_counts()

Series([], Name: prior1, dtype: int64)

In [59]:
ny_election_data['prior2'].value_counts()

Series([], Name: prior2, dtype: int64)

In [60]:
ny_election_data['cand'].value_counts()

scattering                362
seward, james l.           69
lopez, peter d.            55
amedore, george a. jr.     51
little, elizabeth o.       42
                         ... 
meyer, mindy                1
langdon, andrew p.          1
koon, david r.              1
burke, richard e.           1
sweeney, michael j.         1
Name: cand, Length: 770, dtype: int64

In [61]:
ny_election_data['etype'].value_counts()

g                4248
dp                245
rp                181
conservativep     151
indepp            144
workp             136
greenp              5
Name: etype, dtype: int64

Prior columns are useless for this segment of data though incumbency (expr column) is going to be a huge indicator obviously. For this research I'm going to limit to general elections. Then I'm going to need to combine candidates due to NY's fusion voting system.

In [64]:
ny_election_data = ny_election_data[ny_election_data['etype'] == 'g']
ny_election_data

,year,sab,ddez,sen,dname,dno,dtype,etype,cando,cand,party,prior1,prior2,exper,vote,outcome
238120,2012,ny,1,0,NaN,1.0,1.0,g,NaN,"thiele, fred w. jr.",democrat,NaN,NaN,inc,29644.0,w
238121,2012,ny,1,0,NaN,1.0,1.0,g,NaN,"thiele, fred w. jr.",workingfamilies,NaN,NaN,inc,4142.0,w
238122,2012,ny,1,0,NaN,1.0,1.0,g,NaN,"thiele, fred w. jr.",independent,NaN,NaN,inc,4219.0,w
238123,2012,ny,2,0,NaN,2.0,1.0,g,NaN,"deegan, nicholas f.",democrat,NaN,NaN,none,18159.0,l
238124,2012,ny,2,0,NaN,2.0,1.0,g,NaN,"losquadro, daniel p.",independent,NaN,NaN,inc,1941.0,w
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248553,2016,ny,63,1,NaN,63.0,1.0,g,Timothy M. Kennedy,"kennedy, timothy m.",democrat,NaN,NaN,inc,78674.0,w
248554,2016,ny,63,1,NaN,63.0,1.0,g,Timothy M. Kennedy,"kennedy, timothy m.",wep,NaN,NaN,inc,1191.0,w
248555,2016,ny,63,1,NaN,63.0,1.0,g,Timothy M. Kennedy,"kennedy, timothy m.",indep,NaN,NaN,inc,3751.0,w
248556,2016,ny,63,1,NaN,63.0,1.0,g,Timothy M. Kennedy,"kennedy, timothy m.",working,NaN,NaN,inc,6034.0,w
